<center> <h1>Scientific Machine Learning: Bridging Domain Knowledge and Artificial Intelligence for Sustainable Industrial Transition - Part 2: Hands on
</h1> </center>  

<center> <h3>Vinicius Viena Santana
    </h3>
</center>  

<center> <h3>Norwegian University of Science and Technology, Department of Chemical Engineering, PSE Group, Idelfonso´s lab </h3></center>

**Background**
- Bachelor and Master's degree in industrial engineering from the Federal University of Bahia, Brazil (Supervised by Idelfonso Nogueira). 🎓
- Ph.D. at the University of Porto, Portugal (Supervised by Idelfonso Nogueira, Chris Rackauckas and Ana Ribeiro) 📚

**About me**

Enthusiastic about coding and the Julia programming language ❤️

**Research**
- Currently a Postdoctoral researcher at NTNU, Norway (PSE Group, Idelfonso's lab)
- Research focus: Scientific Machine Learning and AI applied to industrial and chemical engineering 📈

# Modeling in Natural Sciences

Building simple and powerful models is time consuming and expensive 💰. Yet, researchers/engineers dedicate a great fraction of their time to it.

* Models compete for acceptance. Should we favor rigor or simplicity, exactness or usefulness? 

# An engineering problem: Two approaches

A waste treatment plant operates a process where contaminated water flows through a filtration system to remove these impurities. Developing a mathematical model this process is important.


With it, we can answer questions such as:

* **How much time it takes for the outlet concentration to reach a certain value?**

The contaminant binds to the particles and are removed from the water stream.

<center><img src="contaminant.svg" width="450"/></center>

<center><img src="models.png" width="1000"/></center>

How can one build this model? 

* First we need data!
* We propose a model with unknown parameters.
* We fit the data and check the fitting quality.
* Finally, we use our model to control/design our unit. 

## Trying to solve the problem in a complete ''Data Driven" way

The data was collected where flow rate (Q) is 5.0 L/min and inlet concentration (Cin) is 20 mg/L:

In [408]:
train_data = readdlm("observations.csv", ',')
Cin = 20
Q = 5
scatter(train_data[:, 1], train_data[:, 2], xlabel = "time (min)", ylabel = "Concentration (mg/L)", label = "Outlet c")

plot!(train_data[:, 1], ones(size(train_data, 1))*Cin, xlabel = "time (min)",
        ylabel = "Concentration (mg/L)", label = "Inlet c", size = (500, 300), legend = :bottomright)

💡 Ok! I can fit a model with this data. 


Whenever someone needs to estimate the time it takes for the contaminat concentration to reach a certain value, I can do it.  

I will use a neural networks because it can approximate any function. Inputs can be $t$, $Cin$ and $Q$ and my outputs will be the concentration $C$. 

That means I want to find a function $f$ that establish this relationship:

$f: (Cin, Q, t)$ $\rightarrow$ $C$ or better $f$: t $\rightarrow$ $C$

## Hands on coding in Julia

In [445]:
using Lux, Statistics, Optimisers, Zygote, ADTypes, Printf

C = Chain(Dense(1, 50, sigmoid), Dense(50, 50, sigmoid), Dense(50, 1)) #Initializing NN structure

Chain(
    layer_1 = Dense(1 => 50, sigmoid_fast),  # 100 parameters
    layer_2 = Dense(50 => 50, sigmoid_fast),  # 2_550 parameters
    layer_3 = Dense(50 => 1),           # 51 parameters
)         # Total: 2_701 parameters,
          #        plus 0 states.

In [446]:
using Random;
rng = Random.default_rng(222)
pinit, st = Lux.setup(rng, C);

In [456]:
scaled_data = (train_data .- minimum(train_data, dims = 1))./(maximum(train_data, dims = 1) .- minimum(train_data, dims = 1))
    
train_index = Int(round(.7*size(scaled_data, 1)))
data = scaled_data[1:train_index, :];
test_data =  scaled_data[train_index:end, :];

In [457]:
function loss(model, ps, st, data)
    y_pred, st = Lux.apply(model, data[1:1, :], ps, st)
    mse_loss = mean(abs2, y_pred .- data[2:2, :])
    return mse_loss, st, ()
end

loss (generic function with 2 methods)

In [458]:
loss(C, pinit, st, data')

(0.13481271325277333, (layer_1 = NamedTuple(), layer_2 = NamedTuple(), layer_3 = NamedTuple()), ())

In [459]:
opt = Adam(2e-3)
tstate = Lux.Experimental.TrainState(rng, C, opt)
vjp_rule = AutoZygote()

AutoZygote()

In [460]:
function train(tstate::Lux.Experimental.TrainState, vjp, data, epochs)
    for epoch in 1:epochs
        grads, loss_val, stats, tstate = Lux.Experimental.compute_gradients(
            vjp, loss, data, tstate)
        if epoch % 500 == 1 || epoch == epochs
            @printf "Epoch: %3d \t Loss: %.5g\n" epoch loss_val
        end
        tstate = Lux.Experimental.apply_gradients(tstate, grads)
    end
    return tstate
end

tstate = Lux.Experimental.TrainState(rng, C, opt);

In [461]:
tstate = train(tstate, vjp_rule, data', 5000);

Epoch:   1 	 Loss: 0.75181
Epoch: 501 	 Loss: 0.010638
Epoch: 1001 	 Loss: 0.0097962
Epoch: 1501 	 Loss: 0.005072
Epoch: 2001 	 Loss: 0.0003627
Epoch: 2501 	 Loss: 0.00016123
Epoch: 3001 	 Loss: 0.00012634
Epoch: 3501 	 Loss: 0.00010523
Epoch: 4001 	 Loss: 9.4122e-05
Epoch: 4501 	 Loss: 8.8831e-05
Epoch: 5000 	 Loss: 8.6489e-05


In [462]:
ŷ = C(data[:, 1]', tstate.parameters, tstate.states)[1]
ŷ_test = C(test_data[:, 1]', tstate.parameters, tstate.states)[1];

In [463]:
plot(data[:, 1], ŷ', label = "train prediction")
scatter!(data[:, 1], data[:, 2], label = "train data", legend = :bottomright)
plot!(test_data[:, 1], ŷ_test', label = "test prediction")
scatter!(test_data[:, 1], test_data[:, 2], label = "test data", size = (500, 300))

What if we want to have a model to predict what happens when the inlet is less contaminated, i.e., $Cin < 20.0$?

* One may collect more data at several values of $Cin$, and train a bigger model.

### Can we use the same data and physics knowledge to obtain a model that can extrapolate for other contaminant concentration or flow rate?

# Solving the problem with domain knowledge - Universal Differential Equations [Rackauckas et al. (2020)](https://arxiv.org/abs/2001.04385)

~ A small black box into your simple domain specific model ~

* The universal approximator aim to reduce mechanistic model non-explained residuals through small mechanistic extensions.


<center><img src="diagram-20231023.svg" width="700"/></center>

$\textrm{Fluid phase mass balance}$

$V_f \frac{dc}{dt} + M_s \frac{dq}{dt} = Q(c_{\text{in}} - c)$

$\textrm{Solid phase mass balance}$

$\frac{dq}{dt} = f(q, q^*)$

$f(q, q^*) = k_1(q^* - q), k_2(q^* - q)^2, k_3(q^{*2} - q^2)/q,...?$ $\rightarrow$ mechanistic approach

* Why not $f(q, q^*) = \textrm{ANN}(q^*, q,\theta)$ ?

q* is typically expressed as a function of c: $q^* = Q\frac{Kc}{(1 + Kc)}$

# Hands-on UDE implementation

## Defining parameters and Neural Network

In [559]:
begin
# Bed parameters
D_bed = 5*10^-2 #m
L_bed = 5*10^-2 #m
V_bed = 4/3*pi*(D_bed/2)^3
ϵ = 0.45 + randn()*(0.05*0.45)
#ϵ = 0.48
d_p = 2e-3
ρ_p = 1100 #kg/m3
V_p = 4/3*pi*(d_p/2)^3 
V_solid = V_bed*(1-ϵ)
V_gas = V_bed*ϵ
M_solid = ρ_p*V_solid #kg

# Inlet conditions
Q = 5 + randn()*(0.05*5)
Cin = 20 # ppm = mg/L

#Isotherm parameters
qmax = 55.54 #mg/g
k = 1.8 #L/mg
kmass = 0.22 #min^-1
end

0.22

* None of those would help if one go for pure data driven approach

### Creating Neural Network using Lux.jl

<center><img src="neural_net.svg" width="600"/></center>

In [560]:
#Scaling parameter (maximum concentration at solid phase)
qstar_max = qmax*k*Cin/(1 + k*Cin);

In [561]:
#Neural Network with 2 inputs and one of this inputs is a function (q*)
using Lux
∂q∂t = Lux.Chain(x -> x./qstar_max, Lux.Dense(2, 8, tanh), Lux.Dense(8, 10, tanh),
                  Lux.Dense(10, 1)); #Uptake rake

In [562]:
using Random;
rng = Random.default_rng(222)
pinit, st = Lux.setup(rng, ∂q∂t);

In [563]:
∂q∂t([50; 50], pinit, st)[1]

1-element Vector{Float64}:
 -0.15931046575460905

## Defining the UDE model

In [564]:
using DifferentialEquations, DiffEqFlux

In [565]:
using ComponentArrays

In [566]:
function ude(du, u, p, t)
    c = u[1] #Fluid phase concentration
    q = u[2] #Solid phase concentration

    qstar = qmax*k*c/(1.0 + k*c)

    du[1] = Q*(Cin - c)/(V_gas*1e3) - M_solid*1e3*∂q∂t([qstar q]', p, st)[1][1]/(V_gas*1e3) 
    du[2] = ∂q∂t([qstar q]', p, st)[1][1]
end

ude (generic function with 1 method)

Simulating the UDE before training it - randomly initialized parameters.

In [567]:
Δt = train_data[2, 1] - train_data[1, 1]; #1 min
t₀ = train_data[1, 1] #0.0 min
tf = train_data[train_index, 1] #50 min
u0 = [0.0; 0.0];
prob = ODEProblem(ude, u0, (t₀, tf), pinit);
#Note that the ODE has two dependent variables but only of them are measured

In [568]:
sol_ude = solve(prob, AutoVern7(KenCarp5(autodiff = true)), abstol = 1e-7, reltol = 1e-7, saveat = Δt);

In [569]:
plot(sol_ude.t, Array(sol_ude)[1, :], label = "untrained UDE", size = (700, 250))
scatter!(train_data[1:train_index, 1], train_data[1:train_index, 2], xlabel = "time (min)", ylabel = "Concentration (mg/L)", label = "observation")

## Training using only one of the states as training data - c(t)

In [570]:
using SciMLSensitivity, Optimization, OptimizationOptimisers, OptimizationOptimJL, Zygote

In [571]:
function predict(p, states)
    new_prob = remake(prob; p = p)
    sensealg = InterpolatingAdjoint(autojacvec = ReverseDiffVJP(true)) #that is a very important choice to make
    out = Array(solve(new_prob, AutoVern7(KenCarp4(autodiff = false)), abstol = 1e-6, reltol = 1e-6,
    sensealg = sensealg, saveat = Δt))[states, :]; #g = [0, 1] * u
    out
end


predict (generic function with 1 method)

In [572]:
function loss(p)
    y = predict(p, 1) #Call predict function
    ∑e² = sum(abs2, (y .- train_data[1:train_index, 2])/Cin) #squared error
    ∑e²
end

loss (generic function with 2 methods)

### Setting up optimization problem

In [573]:
loss(pinit) #High initial loss

29.493708874501223

In [574]:
adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x,p) -> loss(x), adtype);
optprob = Optimization.OptimizationProblem(optf, ComponentArray(pinit));

In [575]:
losses = Float64[]
callback = function (p, l)
    push!(losses, l)
    if length(losses) % 20 == 0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    return false
end


#25 (generic function with 1 method)

In [576]:
@time res1 = Optimization.solve(optprob, ADAM(0.075), callback = callback, maxiters = 250);

Current loss after 20 iterations: 0.11002935773969906
Current loss after 40 iterations: 0.07633692037702387
Current loss after 60 iterations: 0.018820959090987955
Current loss after 80 iterations: 0.010282066208861962
Current loss after 100 iterations: 0.00845530771777609
Current loss after 120 iterations: 0.007276689738899456
Current loss after 140 iterations: 0.006508718369714101
Current loss after 160 iterations: 0.005840410724441356
Current loss after 180 iterations: 0.005299524168406174
Current loss after 200 iterations: 0.004875550501966513
Current loss after 220 iterations: 0.004530111161347893
Current loss after 240 iterations: 0.0042513514198873935
136.559751 seconds (337.27 M allocations: 400.877 GiB, 18.98% gc time, 1.40% compilation time: 3% of which was recompilation)


In [581]:
test_data = readdlm("test_data.csv", ',');

In [582]:
function predict_full(p, states)
    new_prob = remake(prob; p = p, tspan = (train_data[1, 1], train_data[end, 1]))
    sensealg = InterpolatingAdjoint(autojacvec = ReverseDiffVJP(true)) #that is a very important choice to make
    out = Array(solve(new_prob, AutoVern7(KenCarp4(autodiff = false)), abstol = 1e-6, reltol = 1e-6,
    sensealg = sensealg, saveat = Δt))[states, :]; #g = [0, 1] * u
    out
end

predict_full (generic function with 1 method)

In [583]:
Cin = 20
plot(sol_ude.t[1:train_index], predict_full(res1.u, 1)[1:train_index], label = "Trained UDE", size = (800, 350))
scatter!(train_data[:, 1], train_data[:, 2], xlabel = "time (min)", 
    ylabel = "Concentration (mg/L)", label = "Train data",
legend=:outerbottomright)
plot!(train_data[train_index:end, 1], predict_full(res1.u, 1)[train_index:end], label = "Time extrapolation", size = (800, 350))    

## What if I have a less contaminated stream?

In [584]:
Cin = 10.0
test_prob = remake(prob; p = res1.u, tspan = (0.0, 100.), u0 = [0.0, 0.0])
sol_testprob = Array(solve(test_prob, AutoVern7(KenCarp4(autodiff = true)), abstol = 1e-7, reltol = 1e-7,
        saveat = Δt))[1, :];
plot(0.0:1.0:100.0, sol_testprob, label = "Test set UDE", size = (800, 350))
scatter!(test_data[:, 1], test_data[:, 2], xlabel = "time (min)", 
    ylabel = "Concentration (mg/L)", label = "Test data",
legend=:outerbottomright)

<center><img src="sparsereg.PNG" width="1000"/></center>

<center><img src="sparsereg2.PNG" width="1000"/></center>

## Smoothed Collocation for Fast Two-Stage Training

Training ANNs in ODEs can be slow. Can we speed it up with a warm start?

$ANN(q^*, q) = \frac{dq}{dt}$

How do we estimate dq/dt? First we have to estimate q(t) from c(t).

$V_f \int_{0}^{c}{dc} + M_s \int_{0}^{q}{dq} = Q(c_{\text{in}}t - \int_{0}^{c}{c}dt)$

$q(t) = Q/M_s(c_{\text{in}}t - \int_{0}^{c}{c}dt) - V_f/M_s c(t) $

In [180]:
using DataInterpolations, Integrals

In [187]:
t = train_data[:, 1]
c_exp = train_data[:, 2]
c_itp = AkimaInterpolation(c_exp, t); #Interpolate

In [188]:
integrand(t) = Q/(M_solid*1e3)*c_itp(t)
int_integrand(t) = solve(IntegralProblem((t, p) -> integrand(t), 0.0, t), QuadGKJL()).u
q(t) = Q/(M_solid*1e3)*Cin*t - int_integrand(t) - V_gas/M_solid*c_itp(t)

q (generic function with 1 method)

In [189]:
q_train = q.(t);

We can use Kernel smoothing to estimate dq(t)/dt from q(t)

In [190]:
dq_dt_est, q_smooth = collocate_data(q_train[:, :]', train_data[:, 1], EpanechnikovKernel()); #From DiffEqFlux

([2.535207827004392 2.532016773743905 … 0.010450459894075337 0.008616109173400943], [0.09093161908405922 2.576075130934778 … 53.90842337568725 53.93068327921643])

In [203]:
scatter(t, q_train, xlabel = "time (min)", ylabel = "q (mg/L)", label = "q_exp (estimated)")
plot!(t, q_smooth[:], xlabel = "time (min)",
ylabel = "Concentration (mg/L)", legend=:outerbottomright, label = "Smoothed q", size = (700, 300))

Setting up learning problem

In [192]:
Y = dq_dt_est[:]; #target
X = [qmax*k*c_exp./(1.0 .+ k*c_exp) q_train]'/qstar_max; #Features

2×51 Matrix{Float64}:
  0.176155   0.461967  0.242618  0.234628  …  0.999947  1.00013   0.999656
 -1.5817e-6  0.045919  0.091926  0.138309     0.997175  0.997077  0.997144

In [193]:
ɼ_fast = Lux.Chain(Lux.Dense(2, 8, tanh), Lux.Dense(8, 10, tanh),
                  Lux.Dense(10, 1)); #Uptake rake

In [194]:
using Random;
rng = Random.default_rng(222)
pinit_fast, st_fast = Lux.setup(rng, ɼ_fast);

In [195]:
# Predicting and loss function
function predict_fast(p)
        Y_hat = ɼ_fast(X, p, st_fast)[1][:]
        Y_hat
end

function loss_fast(p)
         Y_hat = predict_fast(p)
         loss = sum(abs2, (Y_hat .- Y))
         return loss
end

loss_fast (generic function with 1 method)

In [196]:
loss_fast(pinit_fast) #testing initial loss

132.19156537433574

Continuing

In [197]:
adtype = Optimization.AutoZygote();
optf_fast = Optimization.OptimizationFunction((x,p) -> loss_fast(x), adtype);
optprob_fast = Optimization.OptimizationProblem(optf_fast, ComponentArray(pinit_fast));

In [198]:
losses_fast = []
callback_fast = function (p, l)
    push!(losses_fast, l)
    if length(losses_fast) % 100 == 0
        println("Current loss after $(length(losses_fast)) iterations: $(losses_fast[end])")
    end
    return false
end

#55 (generic function with 1 method)

In [199]:
@time res_fast = Optimization.solve(optprob_fast, ADAM(1e-2), callback = callback_fast, maxiters = 500);

Current loss after 100 iterations: 4.378522721813006
Current loss after 200 iterations: 1.6674762088658721
Current loss after 300 iterations: 0.5836468296748255
Current loss after 400 iterations: 0.33954833406485174
Current loss after 500 iterations: 0.23219823413579016
 14.679808 seconds (32.51 M allocations: 1.927 GiB, 5.03% gc time, 98.57% compilation time)


In [200]:
p_modified = copy(ComponentArray(pinit))
p_modified.layer_2 = res_fast.u.layer_1
p_modified.layer_3 = res_fast.u.layer_2
p_modified.layer_4 = res_fast.u.layer_3

ComponentVector{Float32,SubArray...}(weight = Float32[-0.3489829 -0.11738045 … -0.38437322 0.68859684], bias = Float32[0.009310756;;])

In [202]:
plot(t, predict(p_modified, 1), label = "First Stage UDE", size = (800, 450))
scatter!(train_data[:, 1], train_data[:, 2], xlabel = "time (min)",  ylabel = "Concentration (mg/L)", label = "observation", legend=:outerbottomright)

## Conclusions


* The Universal Differential Equation (UDE) framework serves as a powerful tool for enhancing the capabilities of basic mathematical models, allowing them to better fit data.
* We explored a straightforward implementation of UDE for estimating uptake kinetics in adsorption processes.
* Additionally, we delved into the application of a fast, two-stage training methodology that employs Kernel smoothing techniques to warm up our ANN parameters.


## Thank you!

Feel free to contact me if you want to know more about it:
    
Email: vinicius.santana@ntnu.no

Code and files are available at: